# Dynamics lineshapes

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import sympy as sp
from ampform.kinematics.phasespace import Kallen
from ampform_dpd.decay import ThreeBodyDecay
from ampform_dpd.dynamics import (
    BlattWeisskopf,
    BreitWignerMinL,
    BuggBreitWigner,
    EnergyDependentWidth,
    FlattéSWave,
    P,
    Q,
)
from ampform_dpd.io import cached
from matplotlib_inline.backend_inline import set_matplotlib_formats

from polarimetry.io import (
    display_doit,
    display_latex,
    mute_ampform_warnings,
    mute_jax_warnings,
)
from polarimetry.lhcb import load_model_parameters, load_three_body_decay
from polarimetry.lhcb.dynamics import formulate_breit_wigner, formulate_flatte_1405
from polarimetry.lhcb.particle import load_particles
from polarimetry.lhcb.symbol import create_meson_radius_symbol
from polarimetry.plot import use_mpl_latex_fonts


def load_parameters(
    decay: ThreeBodyDecay, model_id: int | str = 0
) -> dict[sp.Symbol, complex | float]:
    return load_model_parameters(
        decay=decay,
        filename="../../data/model-definitions.yaml",
        model_id=model_id,
        particle_definitions=particles,
    )


particles = load_particles("../../data/particle-definitions.yaml")
ls_model_id = "Alternative amplitude model obtained using LS couplings"
Sigma = particles["Sigma-"]
K = particles["K-"]
Lambda_c = particles["Lambda_c+"]
p = particles["p"]
pi = particles["pi+"]
mute_jax_warnings()
mute_ampform_warnings()
set_matplotlib_formats("svg")
use_mpl_latex_fonts()

In the following, we consider a decay $0 \xrightarrow{L} (r \xrightarrow{\ell} ij)k$, where the resonance&nbsp;$r$ is produced at the **production vertex** with angular momentum $L$ and decays at the **decay vertex** with angular momentum $\ell$. The meson radii for both vertices are denoted $R_\mathrm{prod}$ and $R_\mathrm{dec}$, respectively.

## Relativistic Breit-Wigner

In [ ]:
s, mr, Γr, mi, mj = sp.symbols("s m_r Gamma_R m_i m_j", nonnegative=True)
m0, mk = sp.symbols("m0 m_k")
R_prod = create_meson_radius_symbol("prod")
R_dec = create_meson_radius_symbol("dec")
L, ell = sp.symbols("L ell", integer=True, nonnegative=True)
display_doit(BreitWignerMinL(s, m0, mk, mr, Γr, mi, mj, ell, L, R_dec, R_prod))

## Bugg Breit-Wigner

In [ ]:
gamma, sA = sp.symbols("gamma s_A")
bugg = BuggBreitWigner(s, mr, Γr, mi, mj, gamma)
sA_expr = mi**2 - mj**2 / 2
display_latex({
    bugg: bugg.evaluate().subs(sA_expr, sA),
    sA: sA_expr,
})

One of the models uses a Bugg Breit–Wigner with an exponential factor:

In [ ]:
q = Q(s, m0, mk)
alpha = sp.Symbol("alpha")
bugg * sp.exp(-alpha * q**2)

## Flatté for S-waves

In [ ]:
Γ1, Γ2, mπ, mΣ = sp.symbols("Gamma1 Gamma2 m_pi m_Sigma")
display_doit(FlattéSWave(s, mr, (Γ1, Γ2), (mi, mj), (mπ, mΣ)))

where, in this analysis, we couple the $\varLambda(1405)$ resonance to the channel $\varLambda(1405) \to \varSigma^-\pi^+$.

## Other definitions

In [ ]:
x, y, z = sp.symbols("x:z")
exprs = [
    EnergyDependentWidth(s, mr, Γr, mi, mj, ell, R_dec),
    BlattWeisskopf(z, ell),
    P(s, mi, mj),
    Q(s, m0, mk),
    Kallen(x, y, z),
]
display_latex({x: x.doit(deep=False) for x in exprs})

## Lineshape visualizations

### Breit–Wigner with form factors

In [ ]:
K892 = particles["K(892)"]
decay = load_three_body_decay([K892.name], particles, min_ls=False)
parameter_defaults = load_parameters(decay, ls_model_id)
exprs = {c: formulate_breit_wigner(c)[0].doit(deep=False) for c in decay.chains}
display_latex(exprs)

In [ ]:
def plot_dynamics(ax, chain) -> None:
    dynamics_expr, parameters = formulate_breit_wigner(chain)
    dynamics_func = cached.lambdify(
        dynamics_expr.doit(),
        parameters=parameters | parameter_defaults,
        backend="jax",
    )
    z = dynamics_func({"sigma1": x**2})
    resonance_mass = dynamics_func.parameters[mass_name]
    phsp_min = pi.mass + K.mass
    phsp_max = Lambda_c.mass - p.mass
    ax.axvline(resonance_mass, c="red", ls="dotted", label=f"${mass_name}$")
    ax.axvspan(x.min(), phsp_min, alpha=0.15, color="gray", label="non-physical")
    ax.axvspan(phsp_max, x.max(), alpha=0.15, color="gray")
    ax.plot(x, np.abs(z), label="abs")
    ax.plot(x, z.imag, label="imag")
    ax.plot(x, z.real, label="real")
    title = Rf"$L={chain.incoming_ls.L}, \ell={chain.outgoing_ls.L}$"
    ax.set_title(title, y=0.86)
    ax.set_xlim(x.min(), x.max())
    ax.set_yticks([])


mass_name = f"m_{{{K892.latex}}}"
x = np.linspace(0.5, 1.5, num=300)
fig, axes = plt.subplots(
    figsize=(6, 2 * len(decay.chains)),
    nrows=len(decay.chains),
    sharex=True,
)
fig.patch.set_facecolor("none")
for ax in axes:
    ax.patch.set_facecolor("none")
    ax.spines["bottom"].set_position("zero")
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
fig.suptitle(f"Breit–Wigner for ${K892.latex}$", y=0.96)
for ax, chain in zip(axes, decay.chains, strict=True):
    plot_dynamics(ax, chain)
axes[2].legend(
    bbox_to_anchor=(1.03, 1.03),
    framealpha=1,
    handlelength=1,
    handletextpad=0.5,
    loc="upper right",
)
fig.tight_layout()
plt.show()

### Flatté for $\varLambda(1405)$

In [ ]:
L1405 = particles["L(1405)"]
decay = load_three_body_decay([L1405.name], particles, min_ls=False)
display_latex(decay.chains)

In [ ]:
exprs = [formulate_flatte_1405(c)[0] for c in decay.chains]
display_latex(exprs)

In [ ]:
def plot_dynamics(ax, chain) -> None:
    dynamics_expr, parameters = formulate_flatte_1405(chain)
    dynamics_func = cached.lambdify(
        dynamics_expr.doit(),
        parameters=parameters,
        backend="jax",
    )
    z = dynamics_func({"sigma2": x**2})
    resonance_mass = dynamics_func.parameters[mass_name]
    ax.plot(x, np.abs(z), label="Abs $F$")
    ax.plot(x, z.imag, label="Im $F$")
    ax.plot(x, z.real, label="Re $F$")
    phsp_min = K.mass + p.mass
    phsp_max = Lambda_c.mass - pi.mass
    ax.axvline(Sigma.mass + pi.mass, c="black", ls="dashed", label=R"$m_\Sigma+m_\pi$")
    ax.axvline(resonance_mass, c="red", ls="--", label=f"${mass_name}$")
    ax.axvspan(x.min(), phsp_min, alpha=0.15, color="gray", label="non-physical")
    ax.axvspan(phsp_max, x.max(), alpha=0.15, color="gray")
    title = Rf"$L={chain.incoming_ls.L}, \ell={chain.outgoing_ls.L}$"
    ax.set_title(title, y=0.82)
    ax.set_xlim(x.min(), x.max())
    ax.set_yticks([])


mass_name = f"m_{{{L1405.latex}}}"
x = np.linspace(1.3, 2.3, num=300)
fig, axes = plt.subplots(figsize=(6, 4), nrows=2, sharex=True)
fig.patch.set_facecolor("none")
for ax in axes:
    ax.patch.set_facecolor("none")
    ax.spines["bottom"].set_position("zero")
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
fig.suptitle(f"Flatté for ${L1405.latex}$", y=0.94)
ax0, ax1 = axes
plot_dynamics(ax0, decay.chains[0])
plot_dynamics(ax1, decay.chains[1])
ax1.legend(
    bbox_to_anchor=(1.08, 1.02),
    framealpha=1,
    handlelength=1,
    handletextpad=0.5,
    loc="upper right",
)
fig.tight_layout()
plt.show()